In [ ]:
from Ks import *
import pickle

# load data
f = open("data.pickle", "rb")
data = pickle.load(f)

In [ ]:
import numpy as np

features = []
labels = []

for i in data:
    tmp = [i.bigCategory, i.day, i.goal, i.backers]
    features.append(tmp)
    labels.append(i.pleged)

X = np.array(features)
y = np.array(labels)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, normalize

# encode category
category = X[:, [0]]
enc = OneHotEncoder(handle_unknown='ignore')
category = enc.fit_transform(category).toarray()

In [ ]:
# max normalize
d = X[:, [1,2,3]]
d = normalize(d, axis=0, norm="max")

In [ ]:
X = np.concatenate((category, d), axis=1)

In [ ]:
from sklearn.model_selection import KFold

# K-fold cross validation
kf = KFold(n_splits=5)
kf.get_n_splits(X)

In [ ]:
from sklearn.neighbors import NearestNeighbors
import pickle as pkl
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

best_acc = 0

for i in range(10, 20):

    for j, [train_index, test_index] in enumerate(kf.split(X)):
        X_train = X[train_index, :]
        y_train = y[train_index]

        X_test = X[test_index, :]
        y_test = y[test_index]

        print("Building on K={} fold={}".format(i, j))
        neigh = NearestNeighbors(n_neighbors=i)
        neigh.fit(X_train)

        print("Testing...")
        n = neigh.kneighbors(X_test, i, return_distance=False)
        pledge_predict = []
        state_predict = []
        actual_state = []

        for inx, nbrs in enumerate(n):
            nbrPlege = y_train[nbrs]
            p = np.mean(nbrPlege)
            pledge_predict.append(p)
            goal = data[test_index[inx]].goal
            state = data[test_index[inx]].state
            actual_state.append(state)

            if (p >= goal and state) or (p < goal and not state):
                state_predict.append(True)
            else:
                state_predict.append(False)
        
        RMSE = np.sqrt((np.square(pledge_predict - y_test).mean(axis=0)))
        tn, fp, fn, tp = confusion_matrix(state_predict, actual_state).ravel()
        acc = accuracy_score(state_predict, actual_state)
        
        print("           Positive       Negative")
        print("Positive     {}             {}".format(tn, fp))
        print("Negative     {}             {}".format(fn, tp))

        print(classification_report(state_predict, actual_state))
        print("Accuracy: {}".format(acc))
        print("RMSE: {}".format(RMSE))

        if acc > best_acc:
            # save best classifier
            clf = open("KNN_clf.pkl", "wb")
            pkl.dump(neigh, clf)
            clf.close()
    